In [22]:
using Flux
using Statistics
using Random
using Zygote
include("DenseNTK.jl")
include("FastNTKMethods.jl")
# Your data
y = [10, 21, 30, 30, 29, 40, 33, 45]





x = hcat(range(1/8,stop=1,step=1/8)...)


data = [(x,y)]





1-element Vector{Tuple{Matrix{Float64}, Vector{Int64}}}:
 ([0.125 0.25 … 0.875 1.0], [10, 21, 30, 30, 29, 40, 33, 45])

In [28]:
model = Chain(DenseNTK(1=>10,relu),DenseNTK(10=>1))|>f64

K = kernel(model,x,false,3)
eig = eigen(K).values
Loss(model,x,y) = Flux.mse(y,model(x))
# for epoch = 1:10000
#     Flux.train!(Loss,model,data,Descent(1/eig[end]))
# end

Loss(model,x,y)


DimensionMismatch: DimensionMismatch: loss function expects size(ŷ) = (8,) to match size(y) = (1, 8)

In [8]:
new_sequence = normalized_data[end-sequence_length+1:end]

# Make the prediction
prediction = model(new_sequence)
forecasted_value = prediction[1] * std(data) + mean(data)  # De-normalize the prediction

println("Forecasted Value: $forecasted_value")

Forecasted Value: 33.17176157831794


┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(3 => 10, relu)
│   summary(x) = 3-element Vector{Float64}
└ @ Flux C:\Users\trist\.julia\packages\Flux\jgpVj\src\layers\stateless.jl:60


In [9]:
# Generate multiple predictions for statistical analysis
num_predictions = 100  # Number of predictions to generate
predictions = []

for _ in 1:num_predictions
    # You can add some randomness to the sequence to simulate different scenarios
    perturbed_sequence = new_sequence .+ 0.01 .* randn(length(new_sequence))
    pred = model(perturbed_sequence)
    push!(predictions, pred[1] * std(data) + mean(data))  # De-normalize
end

max_prediction = maximum(predictions)
min_prediction = minimum(predictions)
std_prediction = std(predictions)

println("Maximum Forecasted Value: $max_prediction")
println("Minimum Forecasted Value: $min_prediction")
println("Standard Deviation of Forecasted Values: $std_prediction")


Maximum Forecasted Value: 33.26842236406576
Minimum Forecasted Value: 33.066566838108685
Standard Deviation of Forecasted Values: 0.045574032068410396
